In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder , StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

In [ ]:
from sklearn.svm import SVR
from sklearn.linear_model import Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import ExtraTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor
from sklearn.decomposition import PCA

In [ ]:
pip show pandas


Name: pandas
Version: 1.5.3
Summary: Powerful data structures for data analysis, time series, and statistics
Home-page: https://pandas.pydata.org
Author: The Pandas Development Team
Author-email: pandas-dev@python.org
License: BSD-3-Clause
Location: /usr/local/lib/python3.10/dist-packages
Requires: numpy, python-dateutil, pytz
Required-by: altair, arviz, bokeh, bqplot, cmdstanpy, cufflinks, datascience, db-dtypes, dopamine-rl, fastai, geemap, geopandas, google-colab, gspread-dataframe, holoviews, mizani, mlxtend, pandas-datareader, pandas-gbq, panel, plotnine, prophet, pymc, seaborn, sklearn-pandas, statsmodels, vega-datasets, xarray, yfinance


In [ ]:
pip show scikit-learn

Name: scikit-learn
Version: 1.2.2
Summary: A set of python modules for machine learning and data mining
Home-page: http://scikit-learn.org
Author: 
Author-email: 
License: new BSD
Location: /usr/local/lib/python3.10/dist-packages
Requires: joblib, numpy, scipy, threadpoolctl
Required-by: fastai, imbalanced-learn, librosa, mlxtend, qudida, sklearn-pandas, yellowbrick


Note that we have took the dataset which is not properly preprocessed (i.e one hot encoded etc) reason for this is that we  will be doing all the preprocessing like scaling and catgorical column treament in single pipeline



In [ ]:
df = pd.read_csv("/content/drive/MyDrive/campus x /Capstone Project /gurgaon_properties_post_feature_selection_v2.csv")

In [ ]:
df.head()

,price,sector,property_type,built_up_area,bedRoom,bathroom,balcony,agePossession,servant room,store room,furnishing_type,luxury_category,floor_category
0,2.99,sector 67,flat,2200.000,3.0,3.0,3,relatively new,1.0,0.0,0.0,Low,Medium Floor
1,0.58,sohna road,flat,817.159,2.0,2.0,2,relatively new,0.0,0.0,1.0,Medium,Medium Floor
2,0.95,sector 71,flat,1192.836,2.0,2.0,0,moderately old,0.0,0.0,1.0,Low,Medium Floor
3,1.50,sector 43,flat,1385.000,3.0,3.0,3+,old property,0.0,1.0,0.0,High,Medium Floor
4,10.00,sector 15,house,4491.218,5.0,5.0,2,old property,0.0,0.0,1.0,Low,Low Floor


In [ ]:
# 0 -> unfurnished
# 1 -> semifurnished
# 2 -> furnished

In [ ]:
df['furnishing_type'].value_counts()

1.0    2363
0.0    1030
2.0     198
Name: furnishing_type, dtype: int64

In [ ]:
df['furnishing_type'] = df['furnishing_type'].replace({1.0:'semifurnished', 0.0:'unfurnished',2.0:'furnished'})

In [ ]:
df.head()

,price,sector,property_type,built_up_area,bedRoom,bathroom,balcony,agePossession,servant room,store room,furnishing_type,luxury_category,floor_category
0,2.99,sector 67,flat,2200.000,3.0,3.0,3,relatively new,1.0,0.0,unfurnished,Low,Medium Floor
1,0.58,sohna road,flat,817.159,2.0,2.0,2,relatively new,0.0,0.0,semifurnished,Medium,Medium Floor
2,0.95,sector 71,flat,1192.836,2.0,2.0,0,moderately old,0.0,0.0,semifurnished,Low,Medium Floor
3,1.50,sector 43,flat,1385.000,3.0,3.0,3+,old property,0.0,1.0,unfurnished,High,Medium Floor
4,10.00,sector 15,house,4491.218,5.0,5.0,2,old property,0.0,0.0,semifurnished,Low,Low Floor


In [ ]:
#to avoide error in cross_val_score
df.append(df[(df['sector'] == 'sector 37') | (df['sector'] == 'sector 17a')])

<ipython-input-10-6bfd47463c6d>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df.append(df[(df['sector'] == 'sector 37') | (df['sector'] == 'sector 17a')])


,price,sector,property_type,built_up_area,bedRoom,bathroom,balcony,agePossession,servant room,store room,furnishing_type,luxury_category,floor_category
0,2.99,sector 67,flat,2200.000000,3.0,3.0,3,relatively new,1.0,0.0,unfurnished,Low,Medium Floor
1,0.58,sohna road,flat,817.159000,2.0,2.0,2,relatively new,0.0,0.0,semifurnished,Medium,Medium Floor
2,0.95,sector 71,flat,1192.836000,2.0,2.0,0,moderately old,0.0,0.0,semifurnished,Low,Medium Floor
3,1.50,sector 43,flat,1385.000000,3.0,3.0,3+,old property,0.0,1.0,unfurnished,High,Medium Floor
4,10.00,sector 15,house,4491.218000,5.0,5.0,2,old property,0.0,0.0,semifurnished,Low,Low Floor
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3588,0.80,sector 81,flat,610.000000,2.0,2.0,2,under construction,0.0,0.0,semifurnished,Low,Medium Floor
3589,3.50,sector 46,house,1610.350000,4.0,4.0,2,relatively new,0.0,0.0,unfurnished,Low,Low Floor
3590,2.04,sector 102,flat,1648.345817,3.0,3.0,3+,relatively new,0.0,0.0,semifurnished,Medium,Low Floor
2487,0.67,sector 37,flat,1079.590339,2.0,2.0,2,relatively new,0.0,0.0,semifurnished,Low,Medium Floor


In [ ]:
df.isna().sum()

In [ ]:
df = df[~df['floor_category'].isna()]

In [ ]:
df.isna().sum()

In [ ]:
x = df.drop('price', axis = 1)
y = df['price']
y_transformed = np.log1p(y)

we will try all the techniques for categorical column treatment i.e
* OrdinalEncoding
* OneHotEncoding
* Target Encoder



In [ ]:
df['balcony'].value_counts()

#OrdinalEncoding

In [ ]:
columns_to_encode = ['sector', 'property_type', 'balcony','agePossession', 'furnishing_type', 'luxury_category', 'floor_category']

In [ ]:
preprocessor = ColumnTransformer(
    transformers = [
        ('num', StandardScaler(), ['built_up_area', 'bedRoom', 'bathroom', 'servant room', 'store room']),
        ('cat', OrdinalEncoder(), columns_to_encode)
    ]
)

preprocessor

In [ ]:
pipeline = Pipeline(
    [
        ('preprocessor', preprocessor),
        ('Regressor', LinearRegression())
    ]
)

pipeline

In [ ]:
kfold = KFold(n_splits = 10, shuffle = True, random_state = 42)
score = cross_val_score(pipeline, x, y_transformed, cv = kfold, scoring = 'r2')

In [ ]:
score.mean()

In [ ]:
x_train , x_test, y_train, y_test = train_test_split(x, y_transformed, test_size = 0.2, random_state = 42)

In [ ]:
pipeline.fit(x_train, y_train)

In [ ]:
y_predict = pipeline.predict(x_test)

In [ ]:
y_predict = np.expm1(y_predict)

In [ ]:
mean_absolute_error(np.expm1(y_test),y_predict)

we can see we are getting a mean_absolute_error

In [ ]:
#creating  a function so that we can try different ml models

def score_calculator(model_name, model):
  output = []
  output.append(model_name)
  preprocessor = ColumnTransformer(
    transformers = [
        ('num', StandardScaler(), ['built_up_area', 'bedRoom', 'bathroom', 'servant room', 'store room']),
        ('cat', OrdinalEncoder(), columns_to_encode)
    ]
  )

  pipeline = Pipeline(
    [
        ('preprocessor', preprocessor),
        ('model_name', model)
    ]
  )

  kfold = KFold(n_splits = 10, shuffle = True, random_state = 42)
  score = cross_val_score(pipeline, x, y_transformed, cv = kfold, scoring = 'r2')
  output.append(score.mean())
  x_train , x_test, y_train, y_test = train_test_split(x, y_transformed, test_size = 0.2, random_state = 42)
  pipeline.fit(x_train, y_train)
  y_predict = pipeline.predict(x_test)
  y_predict = np.expm1(y_predict)
  mae = mean_absolute_error(np.expm1(y_test),y_predict)
  output.append(mae)
  return output

In [ ]:
model_dict = {
    'linear_reg' : LinearRegression(),
    'svr' : SVR(),
    'ridge' : Ridge(),
    'lasso' : Lasso(),
    'decision_tree' : DecisionTreeRegressor(),
    'random_forest' : RandomForestRegressor(),
    'extra_trees' : ExtraTreeRegressor(),
    'gradient_boosting' : GradientBoostingRegressor(),
    'adaboost' : AdaBoostRegressor(),
    'mlp' : MLPRegressor(),
    'xgboost' : XGBRegressor()
}

In [ ]:
score_calculator("lr", LinearRegression())

In [ ]:
m_op = []
for m_name, model in model_dict.items():
  m_op.append(score_calculator(m_name, model))


In [ ]:
m_op

In [ ]:
pd.DataFrame(m_op, columns = ['name', 'r2', 'mae']).sort_values(by = 'mae', ascending =True)

#Onehotencoding

if we apply ordinal encoding on categorical column then in case of linear models it might cause a loss of information or missinterpret the categorical data because linear model gives more weightage to big number as compare t0 small numbers (i.e if flat =1 and house = 2 then linear model will consider house is better than flat. so here if coefficient for property type is 10 then for flat weightage will be 10 and for house weightage will be 20 in overall calculation. that is why linear model will do prediction wrongly) due to this linear model might missinterpret the categorical data
to avooid this we use onehot encoding where linear model gives the weightage based on presence of particular feature in data .  (coefficicent for each feature)

In [ ]:
#columns_to_encode = ['property_type']
#ohe = ['sector', 'agePossession', 'furnishing_type','luxury_category', 'floor_category', 'balcony']

columns_to_encode = ['property_type']
ohe = ['sector', 'agePossession', 'furnishing_type','luxury_category', 'floor_category', 'balcony']

In [ ]:
preprocessor = ColumnTransformer(
    transformers = [
        ('num', StandardScaler(), ['built_up_area', 'bedRoom', 'bathroom', 'servant room', 'store room']),
        ('cat', OrdinalEncoder(), columns_to_encode),
        ('ohe', OneHotEncoder(drop='first',sparse_output=False), ohe)
    ], remainder='passthrough'
)


In [ ]:
df.head(5)

In [ ]:
pipeline = Pipeline(
    [
        ('preprocessor', preprocessor),
        ('Regressor', LinearRegression())
    ]
)

In [ ]:
pipeline

In [ ]:
score_kfold = KFold(n_splits = 10, random_state=42, shuffle = True)
score = cross_val_score(pipeline,x, y_transformed, cv = kfold, scoring = 'r2')

In [ ]:
score.mean()

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y_transformed, test_size = 0.2, random_state = 42)
pipeline.fit(x_train, y_train)
y_pred = pipeline.predict(x_test)
y_pred = np.expm1(y_pred)

In [ ]:
mean_absolute_error(np.expm1(y_test),y_pred)


In [ ]:
#creating  a function so that we can try different ml models

def score_calculator(model_name, model):
  output = []
  output.append(model_name)
  preprocessor = ColumnTransformer(
      transformers = [
          ('num', StandardScaler(), ['built_up_area', 'bedRoom', 'bathroom', 'servant room', 'store room']),
          ('cat', OrdinalEncoder(), columns_to_encode),
          ('ohe', OneHotEncoder(drop='first',sparse_output=False), ohe)
      ], remainder='passthrough'
  )

  pipeline = Pipeline(
    [
        ('preprocessor', preprocessor),
        ('model_name', model)
    ]
  )

  kfold = KFold(n_splits = 10, shuffle = True, random_state = 42)
  score = cross_val_score(pipeline, x, y_transformed, cv = kfold, scoring = 'r2')
  output.append(score.mean())
  x_train , x_test, y_train, y_test = train_test_split(x, y_transformed, test_size = 0.2, random_state = 42)
  pipeline.fit(x_train, y_train)
  y_predict = pipeline.predict(x_test)
  y_predict = np.expm1(y_predict)
  mae = mean_absolute_error(np.expm1(y_test),y_predict)
  output.append(mae)
  return output

In [ ]:
m_op = []
for m_name, model in model_dict.items():
  m_op.append(score_calculator(m_name, model))

In [ ]:
pd.DataFrame(m_op, columns = ['name', 'r2', 'mae']).sort_values(by = 'mae', ascending =True)

#OneHotEncoding with PCA

In [ ]:
preprocessor = ColumnTransformer(
  transformers = [
      ('num', StandardScaler(), ['built_up_area', 'bedRoom', 'bathroom', 'servant room', 'store room']),
      ('cat', OrdinalEncoder(), columns_to_encode),
      ('ohe', OneHotEncoder(drop='first',sparse_output=False), ohe)
  ], remainder='passthrough'
)

pipeline = Pipeline(
[
    ('preprocessor', preprocessor),
    ('pca', PCA(n_components = 0.95)),
    ('model_name', LinearRegression())
]
)

kfold = KFold(n_splits = 10, shuffle = True, random_state = 42)
score = cross_val_score(pipeline, x, y_transformed, cv = kfold, scoring = 'r2')

x_train , x_test, y_train, y_test = train_test_split(x, y_transformed, test_size = 0.2, random_state = 42)
pipeline.fit(x_train, y_train)
y_predict = pipeline.predict(x_test)
y_predict = np.expm1(y_predict)
mae = mean_absolute_error(np.expm1(y_test),y_predict)

In [ ]:
score.mean(), mae

we can see after applying pca models performance has got decreass

#TargetEncoder

* Just like OrdinalEncoder OR OneHotEncoder TarrgetEncoder is also a encoding technique specially used for High cardinal features.
* It takes the mean of target feature for each category present in column to be encoded.
* Then it repaces the category with the mean value

In [ ]:
pip install category-encoders

In [ ]:
import category_encoders as ce

In [ ]:
columns_to_encode = ['property_type']
ohe = ['agePossession', 'furnishing_type','luxury_category', 'floor_category', 'balcony']

In [ ]:
def score_calculator(m_name, model):

  output = []
  output.append(m_name)
  preprocessor = ColumnTransformer(
    transformers = [
        ('num', StandardScaler(), ['built_up_area', 'bedRoom', 'bathroom', 'servant room', 'store room']),
        ('cat', OrdinalEncoder(), columns_to_encode),
        ('ohe', OneHotEncoder(drop='first',sparse_output=False), ohe),
        ('te', ce.TargetEncoder() , ['sector'])
    ], remainder='passthrough'
  )

  pipeline = Pipeline(
  [
      ('preprocessor', preprocessor),
      ('model_name', model)
  ]
  )

  kfold = KFold(n_splits = 10, shuffle = True, random_state = 42)
  score = cross_val_score(pipeline, x, y_transformed, cv = kfold, scoring = 'r2')
  output.append(score.mean())

  x_train , x_test, y_train, y_test = train_test_split(x, y_transformed, test_size = 0.2, random_state = 42)
  pipeline.fit(x_train, y_train)
  y_predict = pipeline.predict(x_test)
  y_predict = np.expm1(y_predict)
  mae = mean_absolute_error(np.expm1(y_test),y_predict)
  output.append(mae)

  return output

In [ ]:
score.mean(), mae

In [ ]:
model_dict = {
    'linear_reg' : LinearRegression(),
    'svr' : SVR(),
    'ridge' : Ridge(),
    'lasso' : Lasso(),
    'decision_tree' : DecisionTreeRegressor(),
    'random_forest' : RandomForestRegressor(),
    'extra_trees' : ExtraTreeRegressor(),
    'gradient_boosting' : GradientBoostingRegressor(),
    'adaboost' : AdaBoostRegressor(),
    'mlp' : MLPRegressor(),
    'xgboost' : XGBRegressor()
}

In [ ]:
output = []
for m_name, model in model_dict.items():
  output.append(score_calculator(m_name, model))

In [ ]:
output

In [ ]:
pd.DataFrame(output, columns = ['m_name', 'r2', 'mae']).sort_values(by = 'mae', ascending = True)

#we can see random forest and xgboost are consistently performing well.

#HyperparameterTuning

we will be moving forword with random forest and target encider encoding technique

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
param_grid = {
    'model__n_estimators': [50, 100, 150, 200, 250, 500],
    'model__max_features': [None],
    'model__max_depth': [2, 6, 8, 10, 15, 20],
    'model__min_samples_split': [5, 10, 20]
}

preprocessor = ColumnTransformer(
  transformers = [
      ('num', StandardScaler(), ['built_up_area', 'bedRoom', 'bathroom', 'servant room', 'store room']),
      ('cat', OrdinalEncoder(), ['property_type']),
      ('ohe', OneHotEncoder(drop='first',sparse_output=False), ['agePossession', 'furnishing_type','luxury_category', 'floor_category', 'balcony']),
      ('te', ce.TargetEncoder() , ['sector'])
  ], remainder='passthrough'
)

pipeline = Pipeline(
[
    ('preprocessor', preprocessor),
    ('model', RandomForestRegressor())
]
)

kfold = KFold(n_splits = 10, shuffle = True, random_state = 42)
search = GridSearchCV(pipeline, param_grid, cv=kfold, scoring='r2', n_jobs=-1, verbose=4)
search.fit(x, y_transformed)

In [ ]:
final_pipe = search.best_estimator_
final_pipe

In [ ]:
search.best_params_

In [ ]:
search.best_score_

In [ ]:
final_pipe.fit(x, y_transformed)

#Exporting Model

In [ ]:
preprocessor = ColumnTransformer(
  transformers = [
      ('num', StandardScaler(), ['built_up_area', 'bedRoom', 'bathroom', 'servant room', 'store room']),
      ('cat', OrdinalEncoder(), ['property_type']),
      ('ohe', OneHotEncoder(drop='first',sparse_output=False), ['agePossession', 'furnishing_type','luxury_category', 'floor_category', 'balcony']),
      ('te', ce.TargetEncoder() , ['sector'])
  ], remainder='passthrough'
)

In [ ]:
pipeline = Pipeline(
[
    ('preprocessor', preprocessor),
    ('model', RandomForestRegressor(max_depth=20, max_features=None, min_samples_split=5,
                     n_estimators=500))]
)


In [ ]:
pipeline.fit(x, y_transformed)

In [ ]:
import pickle

with open('pipeline.pkl', 'wb') as file:
  pickle.dump(pipeline, file)


with open('df.pkl', 'wb') as file:
  pickle.dump(x, file)

##lets try some predictions

In [ ]:
one_df = pd.DataFrame([list(x.iloc[300,:].values)], columns = x.columns)

In [ ]:
np.expm1(pipeline.predict(one_df))

In [ ]:
y[300]